# Polygon Plasma Bridge utilized for Polymorphs NFT Collection
- This is a notebook that can serve as a guide to explore bridging Polymorph NFTs from Ethereum to Polygon and vice versa.

# Ethereum -> Polygon Steps

0) Deploy all 5 contracts to the corresponding networks. For reference on how to do that see `README.md`. 

1) Call `setFxChildTunnel` on deployed RootTunnel with the address of child tunnel

2) Call `setFxRootTunnel` on deployed ChildTunnel with address of root tunnel

3) Call `setPolymorphContract` on RootTunnel to point to the RootPolymorph address

4) Call `setPolymorphContract` on ChildTunnel to point to the ChildPolymorph address

5) WhiteList `PolyRoot Tunnel` from `PolymorphRoot`

6) WhiteList `PolyChild Tunnel` from `PolymorphChild`

7) Mint new NFT either with call/sendTransaction or bulkBuy()

8) Approve `Root Tunnel Contract` to manage an NFT

9) Call MoveThroughWormhole(tokenId) on `Root Tunnel Contract`

- Go to Polygon network and observe the genome of the NFT that you have bridged. It should have changed :)
    - Note: It may take 10-20 mins in order for your bridge transaction to be validated. 
    - You can observe the events of the fxChild Contract in Mumbai - `0xCf73231F28B7331BBe3124B907840A94851f9f11` as when your transaction gets validated, it will appear there.

In [1]:
const env = require('dotenv').config({path:'../.env'});
const { getDefaultProvider, providers, Wallet, Contract, utils } = require("ethers");

## Declaring addresses of deployed contracts

In [2]:
const POLY_ROOT_ADDRESS = "0x9e950dD2Ac6Cb90D939406e521B3A81C045A5Dc7";

const ROOT_TUNNEL_ADDRESS = "0xC23887Ed467bc6B9dF48d505e3C1A0326d50eA9A";

const POLY_CHILD_ADDRESS = "0x3120E82A86Ff02283670644486FcCd26df305Ebe";

const CHILD_TUNNEL_ADDRESS = "0x86E79AC4a9CC7003Eb8E0EdD5848891aF6A206D0";

const TEST_ERC_20_ADDRESS = "0x4Fb90bc32709d73A5E745B56708C84A6Ad7Ab5C9";

In [3]:
const gasLimit = "0x100000";

## Declaring providers and Signers

In [4]:
const GOERLI_TESTNET = "goerli";

const MUMBAI_TESTNET = "maticmum";

const PROVIDER_GOERLI = getDefaultProvider(GOERLI_TESTNET, {
    alchemy: env.parsed.GOERLI_ALCHEMY_KEY
});

const PROVIDER_MUMBAI = new providers.AlchemyProvider(MUMBAI_TESTNET, env.parsed.MUMBAI_ALCHEMY_KEY);

const SIGNER_GOERLI = new Wallet(env.parsed.PRIVATE_KEY, PROVIDER_GOERLI);

const SIGNER_MUMBAI = new Wallet(env.parsed.PRIVATE_KEY, PROVIDER_MUMBAI);

## Import ABIs

In [5]:
import POLYMORPH_ROOT_ABI from './abis/POLYROOT_ABI.json';

import POLYMORPH_CHILD_ABI from './abis/POLYCHILD_ABI.json';

import TUNNEL_ROOT_ABI from './abis/TUNNEL_ROOT_ABI.json';

import TUNNEL_CHILD_ABI from './abis/TUNNEL_CHILD_ABI.json';

## Instantiating Contracts

In [6]:
const polyRootInst = new Contract(POLY_ROOT_ADDRESS, POLYMORPH_ROOT_ABI, SIGNER_GOERLI);

const tunnelRootInst = new Contract(ROOT_TUNNEL_ADDRESS, TUNNEL_ROOT_ABI, SIGNER_GOERLI);

const polyChildInst = new Contract(POLY_CHILD_ADDRESS, POLYMORPH_CHILD_ABI, SIGNER_MUMBAI);

const tunnelChildInst = new Contract(CHILD_TUNNEL_ADDRESS, TUNNEL_CHILD_ABI, SIGNER_MUMBAI);

## Running Bridge Prerequisites

### Link Root And Child Polymorph Contracts with the respective tunnels

In [7]:
const setContractInRootTx = await tunnelRootInst.setPolymorphContract(
           POLY_ROOT_ADDRESS, {gasLimit : utils.hexlify(gasLimit)
        });

In [8]:
setContractInRootTx

{
  type: 2,
  chainId: 5,
  nonce: 46,
  maxPriorityFeePerGas: BigNumber { _hex: '0x9502f900', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x9502f92c', _isBigNumber: true },
  gasPrice: null,
  gasLimit: BigNumber { _hex: '0x100000', _isBigNumber: true },
  to: '0xC23887Ed467bc6B9dF48d505e3C1A0326d50eA9A',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  data: '0xc8509ec20000000000000000000000009e950dd2ac6cb90d939406e521b3a81c045a5dc7',
  accessList: [],
  hash: '0x892de67ebc49e99ea4d510a4f08e600a7163e48ef7b45e52971d4e3e2d4dd00e',
  v: 0,
  r: '0xa90f32db328177a2224363ab80e2fcee8d7ec442bd248527eab4ea69769e15d0',
  s: '0x47e2a0c34f462499448e9eb02c9033176ea9fbd675ae2d76de55eb71483960b9',
  from: '0x8FcE67537676879Bc5a1B86B403400E1614Bfce6',
  confirmations: 0,
  wait: [Function (anonymous)]
}


In [9]:
const setContractInChildTx = await tunnelChildInst.setPolymorphContract(
           POLY_CHILD_ADDRESS, {gasLimit : utils.hexlify(gasLimit)
        });

In [10]:
setContractInChildTx

{
  type: 2,
  chainId: 80001,
  nonce: 58,
  maxPriorityFeePerGas: BigNumber { _hex: '0x9502f900', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x950a536e', _isBigNumber: true },
  gasPrice: null,
  gasLimit: BigNumber { _hex: '0x100000', _isBigNumber: true },
  to: '0x86E79AC4a9CC7003Eb8E0EdD5848891aF6A206D0',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  data: '0xc8509ec20000000000000000000000003120e82a86ff02283670644486fccd26df305ebe',
  accessList: [],
  hash: '0xd107470f17b878b95277af462af6eef4cc1c618b0ac5a50825100da25ac177c4',
  v: 0,
  r: '0xbb2dcd2aa553b6859287e33c859749afa18c2a1f57a3755e10c72222624e1595',
  s: '0x6b7e684dcffe178fb7570d3da84ee359fb84b032837842ac49db7f9951a9d595',
  from: '0x8FcE67537676879Bc5a1B86B403400E1614Bfce6',
  confirmations: 0,
  wait: [Function (anonymous)]
}


### Whitelist Bridge Addresses

In [11]:
const whiteListTx = await polyRootInst.whitelistBridgeAddress(ROOT_TUNNEL_ADDRESS, true);

========= NOTICE =========
Request-Rate Exceeded  (this message will not be repeated)

The default API keys for each service are provided as a highly-throttled,
community resource for low-traffic projects and early prototyping.

While your application will continue to function, we highly recommended
signing up for your own API keys to improve performance, increase your
request rate/limit and enable other perks, such as metrics and advanced APIs.

For more details: https://docs.ethers.io/api-keys/


In [12]:
whiteListTx

{
  type: 2,
  chainId: 5,
  nonce: 47,
  maxPriorityFeePerGas: BigNumber { _hex: '0x9502f900', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x9502f924', _isBigNumber: true },
  gasPrice: null,
  gasLimit: BigNumber { _hex: '0xb57f', _isBigNumber: true },
  to: '0x9e950dD2Ac6Cb90D939406e521B3A81C045A5Dc7',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  data: '0xab39a3c8000000000000000000000000c23887ed467bc6b9df48d505e3c1a0326d50ea9a0000000000000000000000000000000000000000000000000000000000000001',
  accessList: [],
  hash: '0xc2b178b082d9fb110ceeeb1d563d1b425d6595b9e7b2f6eadcf38c5600fb46f0',
  v: 1,
  r: '0x651be6cf394cfd9f5c33d179acb93e812cc69065e1d5ca67a88a32d6a15146d8',
  s: '0x6c1db5b153be9af726f072ab8fffe9b40679730d4abd4bdbdc9c7daedd499d05',
  from: '0x8FcE67537676879Bc5a1B86B403400E1614Bfce6',
  confirmations: 0,
  wait: [Function (anonymous)]
}


In [13]:
const whiteListTx = await polyChildInst.whitelistBridgeAddress(CHILD_TUNNEL_ADDRESS, true);

In [14]:
whiteListTx

{
  type: 2,
  chainId: 80001,
  nonce: 59,
  maxPriorityFeePerGas: BigNumber { _hex: '0x9502f900', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x95097f36', _isBigNumber: true },
  gasPrice: null,
  gasLimit: BigNumber { _hex: '0xb57e', _isBigNumber: true },
  to: '0x3120E82A86Ff02283670644486FcCd26df305Ebe',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  data: '0xab39a3c800000000000000000000000086e79ac4a9cc7003eb8e0edd5848891af6a206d00000000000000000000000000000000000000000000000000000000000000001',
  accessList: [],
  hash: '0xb28596c9050046b75a92317e25f541c22e04742a80af2169cc481b70f5763e66',
  v: 1,
  r: '0x0f83ae584afd5cd0ab284656c5d71c1fe52c0e9cc0d8a0ab211925a9396904ab',
  s: '0x76ee9df0e2adb826d0a0269419daeb435c16797bc30484450552a2fa3f8633d8',
  from: '0x8FcE67537676879Bc5a1B86B403400E1614Bfce6',
  confirmations: 0,
  wait: [Function (anonymous)]
}


### Link both tunnels each other

In [15]:
const setFxChildTunnelTx = await tunnelRootInst.setFxChildTunnel(CHILD_TUNNEL_ADDRESS);

In [16]:
setFxChildTunnelTx

{
  type: 2,
  chainId: 5,
  nonce: 48,
  maxPriorityFeePerGas: BigNumber { _hex: '0x9502f900', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x9502f922', _isBigNumber: true },
  gasPrice: null,
  gasLimit: BigNumber { _hex: '0xac5c', _isBigNumber: true },
  to: '0xC23887Ed467bc6B9dF48d505e3C1A0326d50eA9A',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  data: '0xaea4e49e00000000000000000000000086e79ac4a9cc7003eb8e0edd5848891af6a206d0',
  accessList: [],
  hash: '0x806bad1d2bfe0779b31d8df92da327a54e729f870a8ed234a563d643eea2b582',
  v: 0,
  r: '0xb4d7e764a8ffb7cc91d9556a5c817a9c1ce87cfd5db57a6bca0756923dc7bd95',
  s: '0x2e0cd57bdd716326569b5013c98180e9e21ab2214bd8a330bbadf7d8ff5578ea',
  from: '0x8FcE67537676879Bc5a1B86B403400E1614Bfce6',
  confirmations: 0,
  wait: [Function (anonymous)]
}


In [17]:
const setFxRootTunnelTx = await tunnelChildInst.setFxRootTunnel(ROOT_TUNNEL_ADDRESS);

In [18]:
setFxRootTunnelTx

{
  type: 2,
  chainId: 80001,
  nonce: 60,
  maxPriorityFeePerGas: BigNumber { _hex: '0x9502f900', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x950977c8', _isBigNumber: true },
  gasPrice: null,
  gasLimit: BigNumber { _hex: '0xac03', _isBigNumber: true },
  to: '0x86E79AC4a9CC7003Eb8E0EdD5848891aF6A206D0',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  data: '0x88837094000000000000000000000000c23887ed467bc6b9df48d505e3c1a0326d50ea9a',
  accessList: [],
  hash: '0x62e11213838c0d4c818d4d8ad6af38bbca61cf2d743b9c03bd3de9c67de2ae9f',
  v: 1,
  r: '0xc1dd0463b5f3f3a5c5447bc98155c35c843a0e09f16cd771d8d04aff85b20b0f',
  s: '0x72921d3e98de88fba3f7f26de8e800833e931da139db708ea10a3d42a80b12f2',
  from: '0x8FcE67537676879Bc5a1B86B403400E1614Bfce6',
  confirmations: 0,
  wait: [Function (anonymous)]
}


## Mint NFTs to PolymorphRoot Contract

`Note:`
- Since this is a V2 burnToMint version of the contracts, start_token_id=10000 and total_supply=10000 too.
- If we now want to mint new tokens, tx will revert with `total supply reached`
- That's why DAO needs to vote on new totalSupply amount

In [26]:
const newMaxSupply = 10500;

In [27]:
const daoTotalSupplyVote = await polyRootInst.setMaxSupply(newMaxSupply, {gasLimit : utils.hexlify(gasLimit)});

In [28]:
daoTotalSupplyVote

{
  type: 2,
  chainId: 5,
  nonce: 50,
  maxPriorityFeePerGas: BigNumber { _hex: '0x9502f900', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x9502f910', _isBigNumber: true },
  gasPrice: null,
  gasLimit: BigNumber { _hex: '0x100000', _isBigNumber: true },
  to: '0x9e950dD2Ac6Cb90D939406e521B3A81C045A5Dc7',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  data: '0x6f8b44b00000000000000000000000000000000000000000000000000000000000002904',
  accessList: [],
  hash: '0xd54b570a78bce1ed0ab7f4419ef0615dedbfc0a5c2cd23091388a03bee5e3f98',
  v: 1,
  r: '0xdf6afccb53c2777f3a7fc562a8eccbb1a3dfcf4fa44e598dc6ec8084ee5d9024',
  s: '0x04809c4f6804b3a45448ae23d64f11d79f3d1be2d20f17c79083bce059467eba',
  from: '0x8FcE67537676879Bc5a1B86B403400E1614Bfce6',
  confirmations: 0,
  wait: [Function (anonymous)]
}


In [19]:
const tokensToBuyAmount = 3;

In [29]:
const bulkBuyTx = await polyRootInst.bulkBuy(tokensToBuyAmount, {value: utils.parseEther("0.06"), gasLimit : utils.hexlify(gasLimit)}); // excess will be returned

In [30]:
bulkBuyTx

{
  type: 2,
  chainId: 5,
  nonce: 51,
  maxPriorityFeePerGas: BigNumber { _hex: '0x9502f900', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x9502f910', _isBigNumber: true },
  gasPrice: null,
  gasLimit: BigNumber { _hex: '0x100000', _isBigNumber: true },
  to: '0x9e950dD2Ac6Cb90D939406e521B3A81C045A5Dc7',
  value: BigNumber { _hex: '0xd529ae9e860000', _isBigNumber: true },
  data: '0xd5a83d3e0000000000000000000000000000000000000000000000000000000000000003',
  accessList: [],
  hash: '0x0533d51c51e65ae8566ebdbabf18054ee63817d74ae647e63d303f8bdab32be6',
  v: 0,
  r: '0xd33b773395d3330548fc9289d291a16671aafd98f01ab4e225b8353604e4c957',
  s: '0x0f23aa54113354f00bf8c66da0e512e7523a6d79f4d5d3ed22e78a2c275b4c8f',
  from: '0x8FcE67537676879Bc5a1B86B403400E1614Bfce6',
  confirmations: 0,
  wait: [Function (anonymous)]
}


In [11]:
const lastTokenId = await polyRootInst.lastTokenId();

In [12]:
lastTokenId

BigNumber { _hex: '0x2713', _isBigNumber: true }


In [34]:
// Should be 10003
lastTokenId.toNumber()

10003


In [35]:
const geneOfLastTokenId = await polyRootInst.geneOf(lastTokenId.toNumber());

In [41]:
geneOfLastTokenId

BigNumber {
  _hex: '0x570d4109f00e9daedbdab84a4829cfe48ab77b5cac60f495dab9781f16b1737e',
  _isBigNumber: true
}


## Approve RootTunnel to manage an NFT

In [42]:
// Approve NFT #10003 (last token)
const approveTx = await polyRootInst.approve(ROOT_TUNNEL_ADDRESS, lastTokenId.toNumber());

In [43]:
approveTx

{
  type: 2,
  chainId: 5,
  nonce: 52,
  maxPriorityFeePerGas: BigNumber { _hex: '0x9502f900', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x9502f910', _isBigNumber: true },
  gasPrice: null,
  gasLimit: BigNumber { _hex: '0xbe99', _isBigNumber: true },
  to: '0x9e950dD2Ac6Cb90D939406e521B3A81C045A5Dc7',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  data: '0x095ea7b3000000000000000000000000c23887ed467bc6b9df48d505e3c1a0326d50ea9a0000000000000000000000000000000000000000000000000000000000002713',
  accessList: [],
  hash: '0xfa5d7474ef0f8886a2207617ad8dc0b1ff9dc8128749f7a17ddffd33ffe61b0d',
  v: 0,
  r: '0xb6eff5032b232c6cf2bb1d2c65a790549041c1a17621665b717609cbc08a1852',
  s: '0x5eb0363cd910764507b07b23575aea2ce92ff2f3649ea988fbc063afcf5c7793',
  from: '0x8FcE67537676879Bc5a1B86B403400E1614Bfce6',
  confirmations: 0,
  wait: [Function (anonymous)]
}


## MoveThroughWormHole Transaction

In [44]:
const moveThroughWormHoleTx = await tunnelRootInst.moveThroughWormhole(lastTokenId.toNumber(), {gasLimit : utils.hexlify(gasLimit)});

In [45]:
moveThroughWormHoleTx

{
  type: 2,
  chainId: 5,
  nonce: 53,
  maxPriorityFeePerGas: BigNumber { _hex: '0x9502f900', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x9502f910', _isBigNumber: true },
  gasPrice: null,
  gasLimit: BigNumber { _hex: '0x100000', _isBigNumber: true },
  to: '0xC23887Ed467bc6B9dF48d505e3C1A0326d50eA9A',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  data: '0xaf57513f0000000000000000000000000000000000000000000000000000000000002713',
  accessList: [],
  hash: '0x257c285a427db3a5864b0ead80ea38a3bf8617bd746ba87f3f636ddccd40fa97',
  v: 0,
  r: '0x93445beb087ff2f31f5e1cde86a792d189c4edb8593c4fa1054275bcb24e7d1e',
  s: '0x5f8f51516cf26117f77015ce218c3cf90e7ddd88add98aba50f071a33949b9ee',
  from: '0x8FcE67537676879Bc5a1B86B403400E1614Bfce6',
  confirmations: 0,
  wait: [Function (anonymous)]
}


- At this point, ownership of the token should be transferred to the Root Tunnel Contract. Let's check:

In [46]:
const ownerOfLastTokenId = await polyRootInst.ownerOf(lastTokenId.toNumber());

In [47]:
ownerOfLastTokenId === ROOT_TUNNEL_ADDRESS

true


- Now the validation of the moveThroughWormohole transaction can take up to 20 minutes.

## Check the gene of the bridged NFT

In [48]:
const bridgedGeneLastToken = await polyChildInst.geneOf(lastTokenId.toNumber());

In [49]:
bridgedGeneLastToken

BigNumber {
  _hex: '0x570d4109f00e9daedbdab84a4829cfe48ab77b5cac60f495dab9781f16b1737e',
  _isBigNumber: true
}


In [50]:
bridgedGeneLastToken._hex === geneOfLastTokenId._hex

true


In [51]:
const isBridgedNFTVirgin = await polyChildInst.isNotVirgin(lastTokenId.toNumber());

In [52]:
!isBridgedNFTVirgin // reverting because the function is 'isNotVirgin'

true


## Conclusion
- Successfully bridged the token with no loss of information!

# Polygon -> Ethereum Steps

- Note: If a user wants to morph/randomize the genome of his token on Polygon, he should pay the exact value of how much the same action is worth on Ethereum network because 1 MATIC != 1 ETH. That's why Wrapped ETH is used on Polygon.

- The payment is made directly to the DAO Address. It is mandatory the user to approve the PolymorphChild address to spend the desired amount of tokens on this wrapped ETH contract. Otherwise morphing/randomizing transcations will fail on Polygon.

1) Approve ChildTunnel contract to manage the NFT

2) Execute moveThroughWormhole Transaction
    - Copy its txHash
3) Execute `node scripts/burnProof.js txHash` to generate a proof that moveThroughWormhole transacation happened on Polygon

4) Call receiveMessage(proof) on `polymorphRootTunnel` with the generated proof

In [53]:
const maticWethAddress = await polyChildInst.maticWETH();

In [55]:
maticWethAddress

0x4Fb90bc32709d73A5E745B56708C84A6Ad7Ab5C9


### Morph a gene
-  so then we can test whether the bridge would return it to Ethereum with the new information.

In [56]:
const genePosition = 5;

In [57]:
const morphAGeneInPolygon = await polyChildInst.morphGene(lastTokenId.toNumber(), genePosition, {value: utils.parseEther("0.2")});

In [58]:
morphAGeneInPolygon

{
  type: 2,
  chainId: 80001,
  nonce: 62,
  maxPriorityFeePerGas: BigNumber { _hex: '0x9502f900', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x95532f8e', _isBigNumber: true },
  gasPrice: null,
  gasLimit: BigNumber { _hex: '0x01ec06', _isBigNumber: true },
  to: '0x3120E82A86Ff02283670644486FcCd26df305Ebe',
  value: BigNumber { _hex: '0x02c68af0bb140000', _isBigNumber: true },
  data: '0x56a5c92600000000000000000000000000000000000000000000000000000000000027130000000000000000000000000000000000000000000000000000000000000005',
  accessList: [],
  hash: '0xfb95c9b4fa45d0f6564fd33e0dd295765cea8c70f7ca21e17cf16efb54e67c82',
  v: 1,
  r: '0x47ba747c8e9ad8a2afe594f1b79ae57dd118f0fe2eaaf3659986264d18f62b19',
  s: '0x30b68f416de41968627871dcc42db970deaeee6818c072b9981d0ddca211947d',
  from: '0x8FcE67537676879Bc5a1B86B403400E1614Bfce6',
  confirmations: 0,
  wait: [Function (anonymous)]
}


- Check the gene whether it's morphed

In [15]:
const morphedGene = await polyChildInst.geneOf(lastTokenId.toNumber());

In [16]:
morphedGene._hex

0x570d4109f00e9daedbdab84a4829cfe48ab77b5cac60f495dab977aaac5eeb7e


In [61]:
geneOfLastTokenId._hex

0x570d4109f00e9daedbdab84a4829cfe48ab77b5cac60f495dab9781f16b1737e


In [62]:
morphedGene._hex === geneOfLastTokenId._hex

false


- as we can see, genes differ, so we have morphed it successfully ;)

### Approve PolymorphChildTunnel to manage the NFT

In [63]:
const approveTx = await polyChildInst.approve(CHILD_TUNNEL_ADDRESS, lastTokenId.toNumber());

In [64]:
approveTx

{
  type: 2,
  chainId: 80001,
  nonce: 63,
  maxPriorityFeePerGas: BigNumber { _hex: '0x9502f900', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x951a951e', _isBigNumber: true },
  gasPrice: null,
  gasLimit: BigNumber { _hex: '0xbeaf', _isBigNumber: true },
  to: '0x3120E82A86Ff02283670644486FcCd26df305Ebe',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  data: '0x095ea7b300000000000000000000000086e79ac4a9cc7003eb8e0edd5848891af6a206d00000000000000000000000000000000000000000000000000000000000002713',
  accessList: [],
  hash: '0x6c0f8cd45f5666a4f58a75bada1e7eb52a022496c2da7cbc47ca0b01ebb9c515',
  v: 1,
  r: '0xff7ab50dc884f1cdd83a336d52c050d845b88656f928f8928d949620c3faa950',
  s: '0x0fc4b9a0cfee0c58708f8e85ee8a1e7735c5cdd70f1ecba5c7354434192d7a77',
  from: '0x8FcE67537676879Bc5a1B86B403400E1614Bfce6',
  confirmations: 0,
  wait: [Function (anonymous)]
}


### MoveThroughWormHole tx

In [65]:
const moveThroughWormHoleBackTx = await tunnelChildInst.moveThroughWormhole(lastTokenId.toNumber());

In [66]:
moveThroughWormHoleBackTx

{
  type: 2,
  chainId: 80001,
  nonce: 64,
  maxPriorityFeePerGas: BigNumber { _hex: '0x9502f900', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x951896ba', _isBigNumber: true },
  gasPrice: null,
  gasLimit: BigNumber { _hex: '0x0166a2', _isBigNumber: true },
  to: '0x86E79AC4a9CC7003Eb8E0EdD5848891aF6A206D0',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  data: '0xaf57513f0000000000000000000000000000000000000000000000000000000000002713',
  accessList: [],
  hash: '0x3c4156a2366893f69de7426eeff3c0021771521b9c1e8bb173cf56040cdf34d6',
  v: 0,
  r: '0x2e86439cd65e76b5883a322f9ccbefa2f84555a9a305237b9bcb2c867620e69d',
  s: '0x1c80885252f4b92d7eff4bf8fc86dbaa5f22524b1110a99cbaf96981842348ae',
  from: '0x8FcE67537676879Bc5a1B86B403400E1614Bfce6',
  confirmations: 0,
  wait: [Function (anonymous)]
}


- copy tx hash

In [67]:
const bridgeBackHash = moveThroughWormHoleBackTx.hash;

In [68]:
bridgeBackHash

0x3c4156a2366893f69de7426eeff3c0021771521b9c1e8bb173cf56040cdf34d6


### Generate proof
- Switch the kernel to Python3 in order to execute the next command as tslab does not support it(?).

- Note: it's possible for the transaction to take a while before checkpointed.

In [9]:
!node ../scripts/burnProof.js "0x3c4156a2366893f69de7426eeff3c0021771521b9c1e8bb173cf56040cdf34d6"

init called ABIManager { networkName: 'testnet', version: 'mumbai' }
args method [Arguments] { '0': 'getLastChildBlock' }
sending tx with config undefined
Is Checkpointed:  true
args method [Arguments] { '0': 'getLastChildBlock' }
sending tx with config undefined
args method [Arguments] { '0': 'currentHeaderBlock' }
sending tx with config undefined
args method [Arguments] { '0': 'headerBlocks', '1': '362530000' }
sending tx with config undefined
args method [Arguments] { '0': 'headerBlocks', '1': '543790000' }
sending tx with config undefined
args method [Arguments] { '0': 'headerBlocks', '1': '634420000' }
sending tx with config undefined
args method [Arguments] { '0': 'headerBlocks', '1': '679740000' }
sending tx with config undefined
args method [Arguments] { '0': 'headerBlocks', '1': '702400000' }
sending tx with config undefined
args method [Arguments] { '0': 'headerBlocks', '1': '713730000' }
sending tx with config undefined
args method [Arguments] { '0': 'headerBlocks', '1': '71

In [7]:
const proof = "0xf91053842b376290b90100231f07d98a77e02cbb381dfa1f41d8785bdb18c8292ce31e66dbd11243cf33e345155d7720d2c5cb6dd27345a910c021fe7514b7bc4e83995fe3cc6dbe919693adf78249aadac1f23f1967d51c4af84e94684897034901f8dc0467627c45b842fad4a69a92141c75100e3693870ef026450960a01d88370bfeebaafa43b4767ac9184615a5bf68f2be9d48ddde424bd90e7ab39e46fd4d0def7e2c60ef0e0ed984bd87046ad6e6dc903de1af1f189396ad306ce3ce3923e112987a5e98ce8fbeac60224eaade3b811fa9cafab6a717224dbf84dad51a679ccbd61d7235a3a0336ba0c64ee38edb1eb6fae76fff63e5eaa34a51cdeb56a0aff218992bc9bf2f61840189cc0884624bee61a08a4e2ffc2c6f2f7f721df27a2258e835530e3adea13e70d6f39c31c77ecfbb08a0652371afde3d113404f71ba969073c861cfac498513f279fc89129708ecc8d5cb9062202f9061e0183516f3fb9010080000100000000000000400000000000000000000000000000000000000000000100000000000200001000000000000000008000000000000000100100200000000001000000000000000008000000900000000000000000000100000000004000000000020000000000200000000800000000000040000080000010000000000000000000000000000000000000000000000000000080000000000000000000220000000000000000000000001000000000002000000000000000000000004000011002000000000401000100000000000000000000000000500040000020000010000000000000000000000000000000010000000000000000000020100000f90513f89c943120e82a86ff02283670644486fccd26df305ebef884a042ef856c2602f37ce625d252830bed486c5c8e9a4de8aa36cc3d15f304eb662ba00000000000000000000000008fce67537676879bc5a1b86b403400e1614bfce6a00000000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000000000000000000000000000000000000000271380f8db943120e82a86ff02283670644486fccd26df305ebef842a08c0bdd7bca83c4e0c810cbecf44bc544a9dc0b9f265664e31ce0ce85f07a052ba00000000000000000000000000000000000000000000000000000000000002713b880570d4109f00e9daedbdab84a4829cfe48ab77b5cac60f495dab977aaac5eeb7e570d4109f00e9daedbdab84a4829cfe48ab77b5cac60f495dab977aaac5eeb7e00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002f89c943120e82a86ff02283670644486fccd26df305ebef884a08c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925a00000000000000000000000008fce67537676879bc5a1b86b403400e1614bfce6a00000000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000000000000000000000000000000000000000271380f89c943120e82a86ff02283670644486fccd26df305ebef884a0ddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3efa00000000000000000000000008fce67537676879bc5a1b86b403400e1614bfce6a00000000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000000000000000000000000000000000000000271380f901199486e79ac4a9cc7003eb8e0edd5848891af6a206d0e1a08c5261668696ce22758910d05bab8f186d6eb247ceac2af2e82c7dc17669b036b8e0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000027130000000000000000000000008fce67537676879bc5a1b86b403400e1614bfce6570d4109f00e9daedbdab84a4829cfe48ab77b5cac60f495dab977aaac5eeb7e00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000001f9013d940000000000000000000000000000000000001010f884a04dfe1bbbcf077ddc3e01291eea2d5c70c2b422b415d95645b9adcfd678cb1d63a00000000000000000000000000000000000000000000000000000000000001010a00000000000000000000000008fce67537676879bc5a1b86b403400e1614bfce6a0000000000000000000000000be188d6641e8b680743a4815dfa0f6208038960fb8a00000000000000000000000000000000000000000000000000000a7005cdfa800000000000000000000000000000000000000000000000000049e6e3f2d98bd16000000000000000000000000000000000000000000000e8ce904a5bd087be04d000000000000000000000000000000000000000000000000049dc73ed0b91516000000000000000000000000000000000000000000000e8ce9054cbd655b884db908d3f908d0f891a03fefb55df6781070c1c80b656a98a2f874f6a4ffd5836842fa7ab5a3f6272f73a02cf79fe9332c9f0e9b321c2a9054bc5ed8696cb0368bd4421b5e355f12054e09a05019548085a2772a70c627297c15471c9c4b5fd46a38e64ece8bbaaa095652048080808080a0902291b75bb9cf6bf126629fa4a29ef061ffa070175e893142760a60518a85ea8080808080808080f90211a0e82b634dc80584e92fa4d749b263803c368b500602611d6e0ba4dcce2a72179ea01cb16990c15d66061af242aae1a3695331278d2b4bd1427ad72f33a471db631aa075dcad89fbde8b9d6640a90f8693135add896f4bb49c52eea85640c669952144a0780ad07f2ad1979bc70d07efefeaa04b203d85415f1e4d78f04c4bc7ef48b9baa0a915d7d820baa4ba7dbe0586d7f46c45518472862a6039b259de248f35d5c3e4a0c3a5d17051f3daf48a712ec8c8f5b567f08816d9ad35c44eafafd7207b09d929a03835a0d510a7411272bb83c4ec52c31789a04ebfa84c370329d8017c694ae0c9a00859c85e4f043d2dc10cc605c5e70622213c09392a080f468dcf0f713e08fb4aa051d77e5a76c239d42ffcdab03fda5cd8ce6b1a2b4eab6178819556a5a1ef5720a0da3223074027a47ac126a619717fba751b5619b67e787bb724b4f58c4999a8d2a09a6a68b0adf8375e5c6528ac8f768198910a5946950e5889c45ce831e7f1882fa00be9d70ef30e027fd80bd1229cb1cb29c5cdfcc82e6ca830917ef799872f1513a083b7ed29a3016e70def0b8f2e599fa5bbad1fda84bf776eacf9580be03832fdda0e1a0387332519fa6050d1f92986f1beeb95aaea3f73d4e5ee803fc466f16e8e0a014644e8edbae9d08bb36cd7759b9be0bc9831ee6bed99ba191a9be24ef23905ea011d0395899aad7bf3ee3aa42d83f59ff84fe6135de4f25068a298931ca1bb97f80f9062620b9062202f9061e0183516f3fb9010080000100000000000000400000000000000000000000000000000000000000000100000000000200001000000000000000008000000000000000100100200000000001000000000000000008000000900000000000000000000100000000004000000000020000000000200000000800000000000040000080000010000000000000000000000000000000000000000000000000000080000000000000000000220000000000000000000000001000000000002000000000000000000000004000011002000000000401000100000000000000000000000000500040000020000010000000000000000000000000000000010000000000000000000020100000f90513f89c943120e82a86ff02283670644486fccd26df305ebef884a042ef856c2602f37ce625d252830bed486c5c8e9a4de8aa36cc3d15f304eb662ba00000000000000000000000008fce67537676879bc5a1b86b403400e1614bfce6a00000000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000000000000000000000000000000000000000271380f8db943120e82a86ff02283670644486fccd26df305ebef842a08c0bdd7bca83c4e0c810cbecf44bc544a9dc0b9f265664e31ce0ce85f07a052ba00000000000000000000000000000000000000000000000000000000000002713b880570d4109f00e9daedbdab84a4829cfe48ab77b5cac60f495dab977aaac5eeb7e570d4109f00e9daedbdab84a4829cfe48ab77b5cac60f495dab977aaac5eeb7e00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002f89c943120e82a86ff02283670644486fccd26df305ebef884a08c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925a00000000000000000000000008fce67537676879bc5a1b86b403400e1614bfce6a00000000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000000000000000000000000000000000000000271380f89c943120e82a86ff02283670644486fccd26df305ebef884a0ddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3efa00000000000000000000000008fce67537676879bc5a1b86b403400e1614bfce6a00000000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000000000000000000000000000000000000000271380f901199486e79ac4a9cc7003eb8e0edd5848891af6a206d0e1a08c5261668696ce22758910d05bab8f186d6eb247ceac2af2e82c7dc17669b036b8e0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000027130000000000000000000000008fce67537676879bc5a1b86b403400e1614bfce6570d4109f00e9daedbdab84a4829cfe48ab77b5cac60f495dab977aaac5eeb7e00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000001f9013d940000000000000000000000000000000000001010f884a04dfe1bbbcf077ddc3e01291eea2d5c70c2b422b415d95645b9adcfd678cb1d63a00000000000000000000000000000000000000000000000000000000000001010a00000000000000000000000008fce67537676879bc5a1b86b403400e1614bfce6a0000000000000000000000000be188d6641e8b680743a4815dfa0f6208038960fb8a00000000000000000000000000000000000000000000000000000a7005cdfa800000000000000000000000000000000000000000000000000049e6e3f2d98bd16000000000000000000000000000000000000000000000e8ce904a5bd087be04d000000000000000000000000000000000000000000000000049dc73ed0b91516000000000000000000000000000000000000000000000e8ce9054cbd655b884d82001804";

### Call receiveMessage() with the proof

In [8]:
const receiveMessageFromChildTx = await tunnelRootInst.receiveMessage(proof);

========= NOTICE =========
Request-Rate Exceeded  (this message will not be repeated)

The default API keys for each service are provided as a highly-throttled,
community resource for low-traffic projects and early prototyping.

While your application will continue to function, we highly recommended
signing up for your own API keys to improve performance, increase your
request rate/limit and enable other perks, such as metrics and advanced APIs.

For more details: https://docs.ethers.io/api-keys/


In [9]:
receiveMessageFromChildTx

{
  type: 2,
  chainId: 5,
  nonce: 54,
  maxPriorityFeePerGas: BigNumber { _hex: '0x9502f900', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x9502f910', _isBigNumber: true },
  gasPrice: null,
  gasLimit: BigNumber { _hex: '0x072460', _isBigNumber: true },
  to: '0xC23887Ed467bc6B9dF48d505e3C1A0326d50eA9A',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  data: '0xf953cec700000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000001056f91053842b376290b90100231f07d98a77e02cbb381dfa1f41d8785bdb18c8292ce31e66dbd11243cf33e345155d7720d2c5cb6dd27345a910c021fe7514b7bc4e83995fe3cc6dbe919693adf78249aadac1f23f1967d51c4af84e94684897034901f8dc0467627c45b842fad4a69a92141c75100e3693870ef026450960a01d88370bfeebaafa43b4767ac9184615a5bf68f2be9d48ddde424bd90e7ab39e46fd4d0def7e2c60ef0e0ed984bd87046ad6e6dc903de1af1f189396ad306ce3ce3923e112987a5e98ce8fbeac60224eaade3b811fa9cafab6a717224dbf84dad51a679ccbd61d7235a3a0336ba0

In [13]:
const newGeneOfLastTokenId = await polyRootInst.geneOf(lastTokenId.toNumber());

In [14]:
newGeneOfLastTokenId

BigNumber {
  _hex: '0x570d4109f00e9daedbdab84a4829cfe48ab77b5cac60f495dab977aaac5eeb7e',
  _isBigNumber: true
}


In [17]:
morphedGene._hex === newGeneOfLastTokenId._hex

true


- Also, ownership of the NFT should now be the user address

In [18]:
const ownerOfLastTokenId = await polyRootInst.ownerOf(lastTokenId.toNumber());

In [19]:
ownerOfLastTokenId

0x8FcE67537676879Bc5a1B86B403400E1614Bfce6


## Conclusion

- Successfully bridged back the morphed NFT with no loss of information about the new gene!